In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
plt.style.use('seaborn')
%matplotlib inline

# Reading csv files

In [ ]:
raised_bp_df = pd.read_csv('C:\#PoL\Code\#Python\charting\Week 4\raisedpb.csv')

In [ ]:
raised_bp_df.head()

In [ ]:
cols = confirmados_df.keys()

Se obtienen todas las fechas con casos confirmados

In [ ]:
from datetime import date
prim_dia = date(2020, 1, 22)
ult_dia = date(2020, 11, 24)
# ult_dia = date(int(input('Ingrese año:')),int(input('Ingrese mes:')),int(input('Ingrese día:')))
rango = ult_dia - prim_dia
dias = rango.days + 4

In [ ]:
#El método DataFrame.loc [] es un método de Pandas que solo toma etiquetas de índice y devuelve una fila o un dataframe(en este caso un df)
confirmados = confirmados_df.loc[:, cols[4]:cols[dias]]

In [ ]:
fechas = confirmados.keys()
casos_brasil = []

In [ ]:
for i in fechas:
    confirmados_sum = confirmados[i].sum()
    casos_brasil.append(confirmados_df[confirmados_df['Country/Region']=='Brazil'][i].sum())

Se obtiene el incremento diario



In [ ]:
def incremento_diario(data):
    d = [] 
    for i in range(len(data)):
        if i == 0:
            d.append(data[0])
        else:
            d.append(data[i]-data[i-1])
    return d

# casos confirmados
brasil_incremento_diario = incremento_diario(casos_brasil)

In [ ]:
dias_desde_1_22 = np.array([i for i in range(len(fechas))]).reshape(-1, 1)
casos_brasil = np.array(casos_brasil).reshape(-1, 1)
brasil_incremento_diario = np.array(brasil_incremento_diario).reshape(-1, 1)

In [ ]:
# Create scatter plot
fig, ax = plt.subplots(figsize=(10,5)) #crea una figura

for fecha, dia, in zip(fechas, brasil_incremento_diario):
    x, y = fecha, dia
    ax.scatter(x,y , alpha=0.8, edgecolors='none', s=30, color= 'r')
plt.xticks(range(0, len(fechas), 30), rotation=45)
plt.title('Casos de Coronavirus por día en Brasil')
plt.ylabel("Casos")
plt.xlabel("Fecha")
ax.grid(False)
plt.legend(loc=0) #localizaciones predeterminadas 0 es con el minimo overlap de los puntos
plt.show()


Se convierten los enteros en "datetime" para mejor visualizacion

---



In [ ]:
comienzo = '1/22/2020'
comienzo_date = datetime.datetime.strptime(comienzo, '%m/%d/%Y')
dias_desde_1_22_fechas = []
for i in range(len(dias_desde_1_22)):
    dias_desde_1_22_fechas.append((comienzo_date + datetime.timedelta(days=i)).strftime('%m/%d/%Y'))

In [ ]:
# modificar ligeramente los datos para que se ajusten mejor al modelo (los modelos de regresión no pueden elegir el patrón)
def muestreo(dias,casos):
    X_train_confirmados, X_test_confirmados, y_train_confirmados, y_test_confirmados = train_test_split(dias, casos, test_size=0.30, shuffle=True, random_state = 42)

    x_train = []
    for i in X_train_confirmados:
        x_train.append(i[0])
    x_train_arr = np.sort(x_train)
    x_train_arr = np.array(x_train_arr).reshape(-1,1)

    x_test = []
    for i in X_test_confirmados:
        x_test.append(i[0])
    x_test_arr = np.sort(x_test)
    x_test_arr = np.array(x_test_arr).reshape(-1,1)

    y_train = []
    for i in x_train_arr:
        y_train.append(int(casos[i][0]))
    y_train_arr = np.array(y_train).reshape(-1,1)
    

    y_test = []
    for i in x_test_arr:
        y_test.append(int(casos[i][0]))
    y_test_arr = np.array(y_test).reshape(-1,1)

    return (x_train_arr,x_test_arr,y_train_arr,y_test_arr)
    
    
x_train,x_test,y_train,y_test = muestreo(dias_desde_1_22,brasil_incremento_diario)

Modelo para predecir el número de casos confirmados. Se usan la máquina de vectores de soporte(vsm), la cresta bayesiana y la regresión lineal en este ejemplo.

In [ ]:
# # use this to find the optimal parameters for SVR
# c = [0.01, 0.1, 1]
# gamma = [0.01, 0.1, 1]
# epsilon = [0.01, 0.1, 1]
# shrinking = [True, False]

# svm_grid = {'C': c, 'gamma' : gamma, 'epsilon': epsilon, 'shrinking' : shrinking}

# svm = SVR(kernel='poly', degree=3)
# svm_search = RandomizedSearchCV(svm, svm_grid, scoring='neg_mean_squared_error', cv=3, return_train_score=True, n_jobs=-1, n_iter=5, verbose=1)
# svm_search.fit(x_train, y_train)


In [ ]:
# svm_search.best_params_

In [ ]:
# svm_confirmados = svm_search.best_estimator_
svm_confirmados = SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1,degree=3, C=0.1)
svm_confirmados.fit(x_train, y_train)
svm_pred = svm_confirmados.predict(dias_desde_1_22)
svm_test_pred = svm_confirmados.predict(x_test)
y_svm_pred = svm_confirmados.predict(x_train)

In [ ]:
#Grafico datos de entrenamiento para encontrar el modelo SVM
plt.figure(12)
plt.figure(figsize=(10,5))
plt.scatter(x_train,y_train,label='train')
plt.plot(x_train,y_svm_pred, label='ajuste', color='r', linewidth=3.0)
plt.title('Encontrar modelo')
plt.legend(['Datos de entrenamiento', 'Ajuste SVM'])
plt.grid()

In [ ]:
print("Coeficiente R^2 de entrenamiento SVM: %.2f"
      % r2_score(y_train, y_svm_pred))

In [ ]:
#Grafico datos de prueba para encontrar el modelo SVM
plt.figure(12)
plt.figure(figsize=(10,5))
plt.scatter(x_test,y_test,label='test')
plt.plot(x_test,svm_test_pred, label='ajuste', color='r', linewidth=3.0)
plt.title('Evaluación de modelo')
plt.legend(['Datos de prueba', 'Ajuste SVM'])
plt.grid()

In [ ]:
print("Coeficiente R^2 de prueba SVM: %.2f"
      % r2_score(y_test, svm_test_pred))

In [ ]:
# transformar nuestros datos para la regresión polinomial
poly = PolynomialFeatures(degree=5)
poly_X_train_confirmados = poly.fit_transform(x_train)
poly_X_test_confirmados = poly.fit_transform(x_test)
poly_dias_desde_1_22 = poly.fit_transform(dias_desde_1_22)

bayesian_poly = PolynomialFeatures(degree=5)
bayesian_poly_X_train_confirmados = bayesian_poly.fit_transform(x_train)
bayesian_poly_X_test_confirmados = bayesian_poly.fit_transform(x_test)
bayesian_poly_dias_desde_1_22 = bayesian_poly.fit_transform(dias_desde_1_22)

In [ ]:
# regresión polinomial
linear_model = LinearRegression(normalize=True, fit_intercept=False)
linear_model.fit(poly_X_train_confirmados, y_train)
test_linear_pred = linear_model.predict(poly_X_test_confirmados)
linear_pred = linear_model.predict(poly_dias_desde_1_22)
y_poly_pred = linear_model.predict(poly_X_train_confirmados)

In [ ]:
print(linear_model.coef_)

In [ ]:
#Grafico datos de entrenamiento para encontrar el modelo Polinomial
plt.figure(12)
plt.figure(figsize=(10,5))
plt.scatter(x_train,y_train,label='train')
plt.plot(x_train,y_poly_pred, label='ajuste', color='r', linewidth=3.0)
plt.title('Lo que quieramo')
plt.legend(['Datos de entrenamiento', 'Ajuste Polinomial'])
plt.grid()

In [ ]:
print("Coeficiente R^2 de entrenamiento polinomial: %.2f"
      % r2_score(y_train, y_poly_pred))

In [ ]:
# plt.plot(y_test)
# plt.plot(test_linear_pred)
# plt.legend(['Datos de prueba', 'Ajuste Polinomial'])
#Grafico datos de entrenamiento para encontrar el modelo SVM
plt.figure(12)
plt.figure(figsize=(10,5))
plt.scatter(x_test,y_test,label='test')
plt.plot(x_test,test_linear_pred, label='ajuste', color='r', linewidth=3.0)
plt.title('Evaluación de modelo')
plt.legend(['Datos de prueba', 'Ajuste SVM'])
plt.grid()

In [ ]:
print("Coeficiente R^2 de prueba polinomial: %.2f"
      % r2_score(y_test, test_linear_pred))

In [ ]:
# regresión polinomial de la cresta bayesiana
tol = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2]
alpha_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
alpha_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
lambda_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
lambda_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
normalize = [True, False]

bayesian_grid = {'tol': tol, 'alpha_1': alpha_1, 'alpha_2' : alpha_2, 'lambda_1': lambda_1, 'lambda_2' : lambda_2, 'normalize' : normalize}

bayesian = BayesianRidge(fit_intercept=False)
bayesian_search = RandomizedSearchCV(bayesian, bayesian_grid, scoring='neg_mean_squared_error', cv=3, return_train_score=True, n_jobs=-1, n_iter=40, verbose=1)
bayesian_search.fit(bayesian_poly_X_train_confirmados, y_train)

In [ ]:
bayesian_search.best_params_

In [ ]:
bayesian_confirmados = bayesian_search.best_estimator_
test_bayesian_pred = bayesian_confirmados.predict(bayesian_poly_X_test_confirmados)
bayesian_pred = bayesian_confirmados.predict(bayesian_poly_dias_desde_1_22)
y_bayesian_pred = bayesian_confirmados.predict(bayesian_poly_X_train_confirmados)


In [ ]:
#Grafico datos de entrenamiento para encontrar el modelo Bayesiano
plt.figure(12)
plt.figure(figsize=(10,5))
plt.scatter(x_train,y_train,label='train')
plt.plot(x_train,y_bayesian_pred, label='ajuste', color='r', linewidth=3.0)
plt.title('Encontrar el modelo')
plt.legend(['Datos de entrenamiento', 'Ajuste Bayesiano'])
plt.grid()

In [ ]:
print("Coeficiente R^2 de entrenamiento Bayesiano: %.2f"
      % r2_score(y_train, y_bayesian_pred))

In [ ]:
#Grafico datos de entrenamiento para encontrar el modelo Bayesiano
plt.figure(12)
plt.figure(figsize=(10,5))
plt.scatter(x_test,y_test,label='test')
plt.plot(x_test,test_bayesian_pred, label='ajuste', color='r', linewidth=3.0)
plt.title('Evaluación de modelo Bayesiano')
plt.legend(['Datos de prueba', 'Ajuste Bayesiano'])
plt.grid()

In [ ]:
print("Coeficiente R^2 de prueba Bayesiana: %.2f"
      % r2_score(y_test, test_bayesian_pred))

In [ ]:
brasil_lista = []
for i in brasil_incremento_diario:
    brasil_lista.append(i[0])
max_value = np.max(brasil_lista)
index = brasil_lista.index(np.max(brasil_lista))
print(index)